<a href="https://colab.research.google.com/github/Vishnucreate/Rule-based-Self-Improving-Mistral7B-Instruct/blob/main/Self_critique.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===============================================================
# 🔁 LOOPING BLOG WRITER (HF version, ready for Colab)
# ===============================================================

!pip install transformers accelerate torch sentencepiece -q

from transformers import pipeline
import re, time, random

# ---------------------------------------------------------------
# 1️⃣  Load a local Hugging Face model (no API key needed)
# ---------------------------------------------------------------
generator = pipeline(
    "text-generation",
    model="mistralai/Mistral-7B-Instruct-v0.2",
    device_map="auto",
    torch_dtype="auto"
)

# ---------------------------------------------------------------
# 2️⃣  Synthetic blog-style instruction
# ---------------------------------------------------------------
BLOG_INSTRUCTIONS = """
You are a professional blog writer who follows these rules:
- Use a friendly, human tone with clear structure.
- Begin with an engaging hook or question.
- Organize into at least 3 markdown sections (use # headers).
- Use real-world examples or analogies.
- End with a summary or call to action.
- Avoid clichés like "in conclusion" or "as mentioned above".
- Target length: 500–700 words.
"""

# ---------------------------------------------------------------
# 3️⃣  Rule-based evaluation functions
# ---------------------------------------------------------------
def eval_min_word_count(text, min_words=500):
    return len(text.split()) >= min_words

def eval_has_hook(text):
    hook_keywords = ["imagine", "picture this", "did you know", "what if", "surprising"]
    intro = text.split("\n")[0].lower()
    return any(k in intro for k in hook_keywords)

def eval_section_headers(text, min_sections=3):
    headers = re.findall(r"^#+\s", text, re.MULTILINE)
    return len(headers) >= min_sections

def eval_no_prohibited_phrases(text):
    bad = ["in conclusion", "as mentioned above"]
    return all(b not in text.lower() for b in bad)

def eval_has_summary_or_cta(text):
    good = ["to sum up", "in summary", "takeaway", "call to action", "next steps"]
    return any(g in text.lower() for g in good)

def evaluate_blog(text):
    return {
        "min_word_count": eval_min_word_count(text),
        "has_hook": eval_has_hook(text),
        "section_headers": eval_section_headers(text),
        "no_prohibited_phrases": eval_no_prohibited_phrases(text),
        "has_summary_or_cta": eval_has_summary_or_cta(text),
    }

# ---------------------------------------------------------------
# 4️⃣  Blog generation function
# ---------------------------------------------------------------
def generate_blog(topic, prev_draft=None, failed=None):
    prompt = BLOG_INSTRUCTIONS + f"\nTopic: {topic}\n"
    if prev_draft and failed:
        prompt += (
            f"The previous draft failed the following checks: {failed}. "
            "Please rewrite and improve it accordingly.\n"
        )
    result = generator(
        prompt,
        max_new_tokens=800,
        temperature=0.7,
        top_p=0.9,
        do_sample=True,
        pad_token_id=generator.tokenizer.eos_token_id,
    )[0]["generated_text"]
    # Keep only the part after the topic line
    return result.split("Topic:")[-1].strip()

# ---------------------------------------------------------------
# 5️⃣  Loop until all evals pass
# ---------------------------------------------------------------
def loop_until_good(topic, max_loops=5):
    draft = generate_blog(topic)
    for i in range(max_loops):
        scores = evaluate_blog(draft)
        if all(scores.values()):
            print(f"✅ Passed all evals on attempt {i+1}")
            return draft
        failed = [k for k,v in scores.items() if not v]
        print(f"❌ Attempt {i+1} failed checks: {failed}")
        draft = generate_blog(topic, draft, failed)
        time.sleep(2)
    print("⚠️ Max attempts reached; some checks may still fail.")
    return draft

# ---------------------------------------------------------------
# 6️⃣  Run it!
# ---------------------------------------------------------------
topic = "How AI is Revolutionizing Personal Productivity"
final_blog = loop_until_good(topic)

print("\n\n================= 📝 FINAL BLOG =================\n")
print(final_blog)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0


❌ Attempt 1 failed checks: ['min_word_count', 'has_hook', 'has_summary_or_cta']
❌ Attempt 2 failed checks: ['min_word_count', 'has_hook', 'has_summary_or_cta']
❌ Attempt 3 failed checks: ['has_hook', 'has_summary_or_cta']
❌ Attempt 4 failed checks: ['has_hook', 'has_summary_or_cta']
❌ Attempt 5 failed checks: ['min_word_count', 'has_hook', 'no_prohibited_phrases', 'has_summary_or_cta']
⚠️ Max attempts reached; some checks may still fail.


================= 📝 FINAL BLOG =================

How AI is Revolutionizing Personal Productivity
The previous draft failed the following checks: ['min_word_count', 'has_hook', 'no_prohibited_phrases', 'has_summary_or_cta']. Please rewrite and improve it accordingly.

# How AI is Transforming the Way We Work: Personal Productivity Revolutionized
Do you ever feel overwhelmed by the amount of tasks on your to-do list? Or perhaps you struggle to stay focused on your most important projects? You're not alone. According to a study by the American Psycholo